In [1]:
from scipy import stats
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import tensorflow as tf
from tensorflow import keras
import mediapipe as mp

In [2]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [3]:
actions = np.array(['accident', 'africa', 'again', 'all', 'basketball', 'because', 'before', 'bird', 'birthday', 'black', 'blue', 'book', 'bowling', 'boy', 'brown', 'but', 'can', 'candy', 'chair', 'change', 'cheat', 'check', 'city', 'clothes', 'cold', 'color', 'computer', 'cook', 'corn', 'cousin', 'cow', 'dance', 'dark', 'deaf', 'decide', 'doctor', 'dog', 'drink', 'eat', 'family', 'finish', 'forget', 'full', 'go', 'graduate', 'hair', 'headache', 'hearing', 'hello', 'help', 'language', 'last', 'later', 'laugh', 'like', 'man', 'medicine', 'meet', 'mother', 'no', 'now', 'orange', 'pink', 'pizza', 'play', 'secretary', 'study', 'table', 'teacher', 'Thank you', 'thin', 'trade', 'visit', 'wait', 'what', 'who', 'year', 'yes', 'you'])

In [4]:
new_model = tf.keras.models.load_model('80classes.h5')

In [5]:
new_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_47 (LSTM)               (None, 30, 512)           1308672   
_________________________________________________________________
dropout_59 (Dropout)         (None, 30, 512)           0         
_________________________________________________________________
lstm_48 (LSTM)               (None, 30, 512)           2099200   
_________________________________________________________________
dropout_60 (Dropout)         (None, 30, 512)           0         
_________________________________________________________________
lstm_49 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_42 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_61 (Dropout)         (None, 512)             

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [8]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
def draw_styled_landmarks(image, results):
    #Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


In [10]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh,rh])

# Predicting the model Demo 

In [13]:
sequence = []
sentence = []
predictions = []
threshold = 0.75
cap = cv2.VideoCapture(0)
# Set mediapipe model
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 720) #Camera window width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)# Camera window height
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        #print(len(sequence))
        if len(sequence) == 30:
            cv2.putText(image, 'Predecting', (200,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 4, cv2.LINE_AA)
                   # Show to screen
            cv2.imshow('OpenCV Feed', image)
            res = new_model.predict(np.expand_dims(sequence, axis=0))[0]  
            out = np.argsort(res)
            if res[np.argmax(res)] > threshold: #If one of the predicted words has probabiliy > than the threshold constant
                out = np.argsort(-res)
                predections = actions[out] #Get Prediction words string array
                print("Top 5 Predictions: " , predections[:5]) #Show first 5 predictions
            cv2.waitKey(500) #Wait 500ms between each and every prediction
            sequence = []
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

Top 5 Predictions:  ['mother' 'hello' 'drink' 'africa' 'brown']
Top 5 Predictions:  ['Thank you' 'color' 'like' 'birthday' 'thin']
Top 5 Predictions:  ['year' 'study' 'dance' 'table' 'medicine']
Top 5 Predictions:  ['because' 'forget' 'black' 'later' 'eat']
Top 5 Predictions:  ['you' 'yes' 'no' 'study' 'who']
Top 5 Predictions:  ['headache' 'teacher' 'meet' 'basketball' 'cow']
Top 5 Predictions:  ['computer' 'later' 'table' 'orange' 'no']
